In [1]:
import pandas as pd
import sqlite3
import numpy as np
import sqlalchemy

In [2]:
context= pd.read_csv("./coronanet_release_allvars_210922.csv", low_memory=False)
context = context[context['entry_type']!="update"]
#context = context[context['correct_type']!="correction"]
context = context[context['init_country_level']=="National"] #ignore regional
context = context[~context['country'].str.contains(",")] #ignore measures implemented by multiple countries at once


context.drop_duplicates(subset ="description", keep = False, inplace = True)

In [3]:
#drop not needed cols
context = context.drop(['record_id', 'policy_id', 'date_end', 'date_announced', 'enforcer', 'dist_index_high_est', 'dist_index_med_est', 'dist_index_low_est', 'dist_index_country_rank', 'institution_status', 'target_country', 'target_geog_level', 'target_region', 'target_province', 'target_city', 'target_other', 'target_who_what', 'target_direction', 'link', 'date_updated', 'recorded_date', 'Rank_FP', 'Score_FP', 'muni_IDC', 'dispersive_IDC', 'constraining_IDC', 'sfi_SFI', 'ti_cpi_TI', 'pop_WDI_PW', 'gdp_WDI_PW', 'growth_WDI_PW', 'lnpop_WDI_PW', 'lngdp_WDI_PW', 'disap_FA', 'polpris_FA', 'latentmean_FA', 'transparencyindex_HR', 'EmigrantStock_EMS', 'type_sub_cat'], axis = 1)
context = context.drop(['tests_units', 'total_vaccinations', 'daily_vaccinations', 'total_vaccinations_per_hundred', 'daily_vaccinations_per_million','confirmed_cases', 'deaths', 'recovered', 'new_tests', 'total_tests', 'total_tests_per_thousand'], axis = 1)
context = context.drop(['province', 'ISO_L2','ISO_A3','ISO_A2', 'city'], axis = 1)
context = context.drop(['update_level', 'travel_mechanism', 'compliance', 'entry_type', 'update_type', 'type_text', 'init_country_level', 'domestic_policy'], axis = 1)

#new in september 2021

context = context.drop(['update_level_var', 'date_end_spec', 'type_new_admin_coop', 'type_vac_cat', 'type_vac_mix', 'type_vac_reg', 'type_vac_amt_scale', 'type_vac_amt_unit', 'type_vac_amt_gov_perc', 'institution_cat', 'institution_conditions', 'target_init_same', 'target_intl_org', 'target_who_gen', 'pdf_link', 'type_vac_purchase', 'type_vac_group', 'type_vac_group_rank', 'type_vac_who_pays', 'type_vac_dist_admin', 'type_vac_loc', 'type_vac_cost_num', 'type_vac_cost_scale', 'type_vac_cost_unit', 'type_vac_cost_gov_perc', 'type_vac_amt_num'], axis = 1)

In [4]:
#bring to shape (similar to memories)
context = context.rename(columns={"date_start": "date", "type": "category", "description": "comment"})
context['date'] = pd.to_datetime(context['date'], format='%Y/%m/%d')
context['date'] = context['date'].dt.date
context["weight"] = 8

In [5]:
#remove "other" from hashtags
context.loc[context["category"].str.contains("Other")]["category"] = np.NaN

#context.loc[context["category"].str.contains("Other")]["category"]

/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [6]:
#remove data that are too early
context = context[context['date'] > np.datetime64('2020-01-20')]

context

,comment,date,country,category,weight
0,"March 6, Afghanistan ""Measures have been taken...",2020-03-06,Afghanistan,Anti-Disinformation Measures,8
1,"March 16, In Afghanistan the Government Media ...",2020-03-16,Afghanistan,Anti-Disinformation Measures,8
2,"Afghanistan, May 28 ""a government livelihood s...",2020-05-28,Afghanistan,Anti-Disinformation Measures,8
3,"Afghanistan, June 28 ""Public awareness campaig...",2020-06-28,Afghanistan,Anti-Disinformation Measures,8
33,"Aug 4, Afghanistan ""has announced that all pri...",2020-08-05,Afghanistan,Closure and Regulation of Schools,8
...,...,...,...,...,...
87595,"On 23 March 2020, Ministry of Health and Child...",2020-03-23,Zimbabwe,Restrictions of Mass Gatherings,8
87603,The Zimbabwe minister of Information and publi...,2020-09-22,Zimbabwe,Restrictions of Mass Gatherings,8
87606,"The Zimbabwean government has, as of 14June 20...",2021-06-14,Zimbabwe,Restrictions of Mass Gatherings,8
87607,"In Zimbabwe, it was announced on the 1st of Ma...",2020-05-01,Zimbabwe,Social Distancing,8


In [7]:
context.loc[context["country"].str.contains("Czechia"), "country"] = "Czech Republic"
context.loc[context["country"].str.contains("Verde"), "country"] = "Cabo Verde"
context.loc[context["country"].str.contains("Egypt"), "country"] = "Egypt, Arab Rep."
context.loc[context["country"].str.contains("Iran"), "country"] = "Iran, Islamic Rep."
context.loc[context["country"].str.contains("Russia"), "country"] = "Russian Federation"
context.loc[context["country"].str.contains("Slovakia"), "country"] = "Slovak Republic"
context.loc[context["country"].str.contains("Syria"), "country"] = "Syrian Arab Republic"
context.loc[context["country"].str.contains("Timor"), "country"] = "Timor-Leste"
context.loc[context["country"].str.contains("United States of America"), "country"] = "United States"
context.loc[context["country"].str.contains("Venezuela"), "country"] = "Venezuela, RB"
context.loc[context["country"].str.contains("Palestine"), "country"] = "West Bank and Gaza" #ooff




In [11]:
context[context['date'] > np.datetime64('2021-06-01')]

,comment,date,country,category,weight
1560,"Argentina. Social, preventive and compulsory i...",2021-10-12,Argentina,Lockdown,8
7464,"30 December, 2020 in Bangladesh, the governmen...",2021-07-01,Bangladesh,Closure and Regulation of Schools,8
7734,"In Bangladesh, the ongoing nationwide lockdown...",2021-08-06,Bangladesh,Lockdown,8
7761,The National Technical Advisory Committee (NTA...,2021-07-25,Bangladesh,Other Policy Not Listed Above,8
8568,"Belgium: As per the summer four-step plan, the...",2021-07-30,Belgium,Restriction and Regulation of Businesses,8
...,...,...,...,...,...
87557,The Zimbabwean government has announced that a...,2021-06-14,Zimbabwe,Quarantine,8
87558,The Zimbabwean government has announced that a...,2021-06-14,Zimbabwe,Quarantine,8
87587,The Zimbabwean government has announced that r...,2021-06-14,Zimbabwe,Restriction and Regulation of Businesses,8
87606,"The Zimbabwean government has, as of 14June 20...",2021-06-14,Zimbabwe,Restrictions of Mass Gatherings,8


In [40]:
#SQLTE
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
context.to_sql(con=sqlite, name='contexts', index=True, if_exists='replace',dtype={"date": str(sqlalchemy.types.DateTime())})

In [32]:


context['category'] = context['category'].replace({'Other Policy Not Listed Above': ''})

#context.loc[context["category"].str.contains("Other")]